<a href="https://colab.research.google.com/github/ambresh1/Colab/blob/main/ML_Translator_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hugging Space Devlopment from file to translated file

In [2]:
!pip install sentencepiece
!pip install torch==1.8.2 torchvision==0.9.2 torchaudio===0.8.2 --extra-index-url https://download.pytorch.org/whl/lts/1.8/cpu
!pip install transformers ipywidgets gradio
!pip install python-docx
!pip install sacremoses


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 7.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/lts/1.8/cpu
     |████████████████████████████████| 169.1 MB 71 kB/s 
     |████████████████████████████████| 13.3 MB 42.8 MB/s 
     |████████████████████████████████| 1.9 MB 52.2 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.1+cu113
    Uninstalling torchvision-0.13.1+cu113:
      Successfully uninstalled torchvision-0.13.1+cu113
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.12.1+cu113
    Uninstalling torchaudio-0.12.1+cu113:
      Successfully uninstall

In [49]:
from transformers import MarianTokenizer, MarianMTModel

def newtranslate(sample_text):
  # print(sample_text)
  # sample_text=str(sample_text)
  #Eng To Rus
  source_file="/content/drive/MyDrive/Transformers Models/Helsinki-NLP/opus-mt-en-ru"
  model_en_ru = MarianMTModel.from_pretrained(source_file)
  tokenizer_en_ru = MarianTokenizer.from_pretrained(source_file)
  batch = tokenizer_en_ru(sample_text, return_tensors="pt")
  # print(batch)
  generated_ids = model_en_ru.generate(**batch, max_new_tokens=2000,) # max_new_tokens=1000
  output_1=tokenizer_en_ru.batch_decode(generated_ids, skip_special_tokens=True)[0]
  # print(output_1) # prints Eng To Rus 
  
  #Rus To Eng
  source_file="/content/drive/MyDrive/Transformers Models/Helsinki-NLP/opus-mt-ru-en"
  model_ru_en = MarianMTModel.from_pretrained(source_file)
  tokenizer_ru_en = MarianTokenizer.from_pretrained(source_file)
  batch = tokenizer_ru_en(output_1, return_tensors="pt")
  # print(batch)
  generated_ids = model_ru_en.generate(**batch, max_new_tokens=2000) #max_new_tokens=1000
  output=tokenizer_ru_en.batch_decode(generated_ids, skip_special_tokens=True)[0]
  # return output # prints Rus To Eng  
  print(output)
  

In [58]:
import nltk
from nltk.tokenize import sent_tokenize 
# import dox as dc 
# nltk.download('punkt')
bt=''' 
Thinking back, she often participated in a lot of storytelling at school. Acting was definitely a piece of cake for her. 

"Ssss, ahhh! My head is hurting so badly. Aiya! Who are you? Why can't I remember? And, who am I?"
"Xiao Yao Zi, what happened? I'm Xiao Mu Zi! Don't scare me!"
Seeing Le Yao Yao's pained expression, Xiao Mu Zi was extremely concerned. His eyes showed that he was anxious and worried. It made Le Yao Yao feel a bit guilty.
After all, this person truly cared about her and she was lying to him.
But she was only half lying. She honestly had no idea whose body she was in, so she had to pretend she lost her memory. Plus, currently she did have a headache.
Earlier she looked around her surroundings and noticed she was in a remote area with a very high wall next to her. She could hear a lot of noises behind the wall. She thought about it, beyond the walls was probably the main streets. Currently, she was most likely in the backyard of a home.
She didn't know why the previous owner was here. Also, she had a huge bump on her head and it was throbbing.
Based on Le Yao Yao's assumptions, the previous owner was probably trying to flip herself over the wall. Clearly, the previous owner failed and fell down. And for some odd reason, her soul entered her body.
Le Yao Yao had no clue what had happened to the previous owner. But she wanted to find out whose body this was, and she will try to stay alive for her sake.
After all, she had transported through time. If she doesn't do some investigating, it will be very tough for her to survive here.
While Le Yao Yao was pondering, suddenly Xiao Mu Zi's expression turned stern. He gravely asked: "Xiao Yao Zi, you're trying to trick me, aren't you? You're just playing dumb. In reality, you haven't lost your memory, am I correct?"

'''
file="/content/drive/MyDrive/Trail Doc of 2000 words.docx"
from docx import Document

doc = Document("/content/drive/MyDrive/Trail Doc of 2000 words.docx")

# for a in doc.paragraphs :
  # newtranslate(a.text)
  # print(a.text)
# sbt=sent_tokenize(bt)

# for a in sbt :
#   newtranslate(a)
# print(sbt)
# newtranslate(bt)
import torch

# position_ids = torch.stack([torch.arange(config.max_position_embeddings) for a in range(val_dataloader.batch_size)]).to(device)

In [55]:
from transformers import MarianMTModel, MarianTokenizer
from nltk.tokenize import sent_tokenize
from nltk.tokenize import LineTokenizer
import math
import torch

if torch.cuda.is_available():  
  dev = "cuda"
else:  
  dev = "cpu" 
device = torch.device(dev)
 
mname = '/content/drive/MyDrive/Transformers Models/opus-mt-en-hi-Trans Model'
tokenizer = MarianTokenizer.from_pretrained(mname)
model = MarianMTModel.from_pretrained(mname)
model.to(device)

lt = LineTokenizer()
batch_size = 8

text_short = ''' He was a demon! Thinking of this, Le Yao Yao became cowardly. '''
text_long = text_short * 30

paragraphs = lt.tokenize(text_long)   
translated_paragraphs = []

for paragraph in paragraphs:
    sentences = sent_tokenize(paragraph)
    batches = math.ceil(len(sentences) / batch_size)     
    translated = []
    for i in range(batches):
        sent_batch = sentences[i*batch_size:(i+1)*batch_size]
        model_inputs = tokenizer(sent_batch, return_tensors="pt", padding=True, truncation=True, max_length=500).to(device)
        with torch.no_grad():
            translated_batch = model.generate(**model_inputs)
        translated += translated_batch
    translated = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    translated_paragraphs += [" ".join(translated)]

translated_text = "\n".join(translated_paragraphs)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1232: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [56]:
print(translated_text)

वह एक राक्षस था! इस बारे में सोचते हुए, ला यारो याो कायर बन गया । वह एक राक्षस था! इस बारे में सोचते हुए, ला यारो याो कायर बन गया । वह एक राक्षस था! इस बारे में सोचते हुए, ला यारो याो कायर बन गया । वह एक राक्षस था! इस बारे में सोचते हुए, ला यारो याो कायर बन गया । वह एक राक्षस था! इस बारे में सोचते हुए, ला यारो याो कायर बन गया । वह एक राक्षस था! इस बारे में सोचते हुए, ला यारो याो कायर बन गया । वह एक राक्षस था! इस बारे में सोचते हुए, ला यारो याो कायर बन गया । वह एक राक्षस था! इस बारे में सोचते हुए, ला यारो याो कायर बन गया । वह एक राक्षस था! इस बारे में सोचते हुए, ला यारो याो कायर बन गया । वह एक राक्षस था! इस बारे में सोचते हुए, ला यारो याो कायर बन गया । वह एक राक्षस था! इस बारे में सोचते हुए, ला यारो याो कायर बन गया । वह एक राक्षस था! इस बारे में सोचते हुए, ला यारो याो कायर बन गया । वह एक राक्षस था! इस बारे में सोचते हुए, ला यारो याो कायर बन गया । वह एक राक्षस था! इस बारे में सोचते हुए, ला यारो याो कायर बन गया । वह एक राक्षस था! इस बारे में सोचते हुए, ला यारो याो कायर बन गया । वह एक राक्

In [1]:
from transformers import MarianMTModel, MarianTokenizer
from nltk.tokenize import sent_tokenize
from nltk.tokenize import LineTokenizer
import math
import torch

if torch.cuda.is_available():  
  dev = "cuda"
else:  
  dev = "cpu" 
device = torch.device(dev)
 
mname = '/content/drive/MyDrive/Transformers Models/opus-mt-en-hi-Trans Model'
tokenizer = MarianTokenizer.from_pretrained(mname)
model = MarianMTModel.from_pretrained(mname)
model.to(device)


def btTranslator(text_long):
  lt = LineTokenizer()
  batch_size = 8
  paragraphs = lt.tokenize(text_long)   
  translated_paragraphs = []

  for paragraph in paragraphs:
      sentences = sent_tokenize(paragraph)
      batches = math.ceil(len(sentences) / batch_size)     
      translated = []
      for i in range(batches):
          sent_batch = sentences[i*batch_size:(i+1)*batch_size]
          model_inputs = tokenizer(sent_batch, return_tensors="pt", padding=True, truncation=True, max_length=500).to(device)
          with torch.no_grad():
              translated_batch = model.generate(**model_inputs)
          translated += translated_batch
      translated = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
      translated_paragraphs += [" ".join(translated)]

  # translated_text = "\n".join(translated_paragraphs)
  
  return translated_paragraphs


ModuleNotFoundError: ignored

In [61]:
btTranslator(bt)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1232: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


('वह सोचने लगी कि स्कूल में हुई कहानी की बहुत - सी बातों में हिस्सा ले रही है । निश्चित रूप से उसके लिए केक का एक टुकड़ा था.\n"Sssss, आह! मेरे सिर बहुत बुरी तरह से चोट लगी है. आइया! तुम कौन हो? मुझे क्यों याद नहीं है? और, मैं कौन हूँ?\n"xiao यारो Zi, क्या हुआ? मैं बोएओ मा Zi हूँ! मुझे डराने मत करो!\nलेओ यारो के शोक की अभिव्यक्\u200dति को देखने पर, जया माओ ज़ियो अति चिन्तित था । उसकी आँखों ने दिखाया कि वह चिन्तित और चिन्तित था । यह लेओ यारो थोड़ा दोषी लग रहा है.\nआख़िरकार, इस व्यक्\u200dति ने उसकी सच्ची परवाह की और वह उससे झूठ बोल रही थी ।\nलेकिन वह केवल आधा झूठ बोल रही थी । उसने ईमानदारी से यह नहीं सोचा था कि वह किस का शरीर अंदर थी, इसलिए उसे अपनी याददाश्\u200dत खोनी पड़ी । इसके अलावा, वर्तमान में वह एक सिरदर्द था.\nइससे पहले उसने अपने आस - पास देखा और देखा कि वह एक दूर - दराज़ इलाके में रहती है, जहाँ उसके पास एक बहुत ही ऊँची दीवार थी । वह दीवार के पीछे शोर का एक बहुत सुन सकता है. उसने इसके बारे में सोचा, दीवारों के बाहर संभवतः मुख्य सड़कें थीं । फिलहाल, वह एक घर के आँगन में सबसे ज़्या

In [9]:
# Def_03 text to docx file 

from transformers import MarianMTModel, MarianTokenizer
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import LineTokenizer
nltk.download('punkt')
import math
import torch
from docx import Document

if torch.cuda.is_available():  
  dev = "cuda"
else:  
  dev = "cpu" 
device = torch.device(dev)
 
mname = '/content/drive/MyDrive/Transformers Models/opus-mt-en-hi-Trans Model'
tokenizer = MarianTokenizer.from_pretrained(mname)
model = MarianMTModel.from_pretrained(mname)
model.to(device)


def btTranslator(text_long):
  files=Document()
  lt = LineTokenizer()
  batch_size = 8
  paragraphs = lt.tokenize(text_long)   
  translated_paragraphs = []

  for paragraph in paragraphs:
      sentences = sent_tokenize(paragraph)
      batches = math.ceil(len(sentences) / batch_size)     
      translated = []
      for i in range(batches):
          sent_batch = sentences[i*batch_size:(i+1)*batch_size]
          model_inputs = tokenizer(sent_batch, return_tensors="pt", padding=True, truncation=True, max_length=500).to(device)
          with torch.no_grad():
              translated_batch = model.generate(**model_inputs)
          translated += translated_batch
      translated = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
      translated_paragraphs += [" ".join(translated)]
      files.add_paragraph(translated)
  # translated_text = "\n".join(translated_paragraphs)
  
  files.save('New_File.docx')
  return translated_paragraphs


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
bt=''' 
Thinking back, she often participated in a lot of storytelling at school. Acting was definitely a piece of cake for her. 

"Ssss, ahhh! My head is hurting so badly. Aiya! Who are you? Why can't I remember? And, who am I?"
"Xiao Yao Zi, what happened? I'm Xiao Mu Zi! Don't scare me!"
Seeing Le Yao Yao's pained expression, Xiao Mu Zi was extremely concerned. His eyes showed that he was anxious and worried. It made Le Yao Yao feel a bit guilty.
After all, this person truly cared about her and she was lying to him.
But she was only half lying. She honestly had no idea whose body she was in, so she had to pretend she lost her memory. Plus, currently she did have a headache.
Earlier she looked around her surroundings and noticed she was in a remote area with a very high wall next to her. She could hear a lot of noises behind the wall. She thought about it, beyond the walls was probably the main streets. Currently, she was most likely in the backyard of a home.
She didn't know why the previous owner was here. Also, she had a huge bump on her head and it was throbbing.
Based on Le Yao Yao's assumptions, the previous owner was probably trying to flip herself over the wall. Clearly, the previous owner failed and fell down. And for some odd reason, her soul entered her body.
Le Yao Yao had no clue what had happened to the previous owner. But she wanted to find out whose body this was, and she will try to stay alive for her sake.
After all, she had transported through time. If she doesn't do some investigating, it will be very tough for her to survive here.
While Le Yao Yao was pondering, suddenly Xiao Mu Zi's expression turned stern. He gravely asked: "Xiao Yao Zi, you're trying to trick me, aren't you? You're just playing dumb. In reality, you haven't lost your memory, am I correct?"

'''

btTranslator(bt)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1232: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


['वह सोचने लगी कि स्कूल में हुई कहानी की बहुत - सी बातों में हिस्सा ले रही है । निश्चित रूप से उसके लिए केक का एक टुकड़ा था.',
 '"Sssss, आह! मेरे सिर बहुत बुरी तरह से चोट लगी है. आइया! तुम कौन हो? मुझे क्यों याद नहीं है? और, मैं कौन हूँ?',
 '"xiao यारो Zi, क्या हुआ? मैं बोएओ मा Zi हूँ! मुझे डराने मत करो!',
 'लेओ यारो के शोक की अभिव्यक्\u200dति को देखने पर, जया माओ ज़ियो अति चिन्तित था । उसकी आँखों ने दिखाया कि वह चिन्तित और चिन्तित था । यह लेओ यारो थोड़ा दोषी लग रहा है.',
 'आख़िरकार, इस व्यक्\u200dति ने उसकी सच्ची परवाह की और वह उससे झूठ बोल रही थी ।',
 'लेकिन वह केवल आधा झूठ बोल रही थी । उसने ईमानदारी से यह नहीं सोचा था कि वह किस का शरीर अंदर थी, इसलिए उसे अपनी याददाश्\u200dत खोनी पड़ी । इसके अलावा, वर्तमान में वह एक सिरदर्द था.',
 'इससे पहले उसने अपने आस - पास देखा और देखा कि वह एक दूर - दराज़ इलाके में रहती है, जहाँ उसके पास एक बहुत ही ऊँची दीवार थी । वह दीवार के पीछे शोर का एक बहुत सुन सकता है. उसने इसके बारे में सोचा, दीवारों के बाहर संभवतः मुख्य सड़कें थीं । फिलहाल, वह एक घर के आ

In [5]:
import docx
def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)

In [14]:
# Def_04 Docx file to translated_Docx file
from transformers import MarianMTModel, MarianTokenizer
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import LineTokenizer
nltk.download('punkt')
import math
import torch
from docx import Document

if torch.cuda.is_available():  
  dev = "cuda"
else:  
  dev = "cpu" 
device = torch.device(dev)
 
mname = '/content/drive/MyDrive/Transformers Models/opus-mt-en-hi-Trans Model'
tokenizer = MarianTokenizer.from_pretrained(mname)
model = MarianMTModel.from_pretrained(mname)
model.to(device)


def btTranslator(docxfile):
  a=getText(docxfile)
  a1=a.split('\n')
  bigtext='''  '''
  for a in a1:
    bigtext=bigtext+'\n'+a
  files=Document()
  lt = LineTokenizer()
  batch_size = 8
  paragraphs = lt.tokenize(bigtext)   
  translated_paragraphs = []

  for paragraph in paragraphs:
      sentences = sent_tokenize(paragraph)
      batches = math.ceil(len(sentences) / batch_size)     
      translated = []
      for i in range(batches):
          sent_batch = sentences[i*batch_size:(i+1)*batch_size]
          model_inputs = tokenizer(sent_batch, return_tensors="pt", padding=True, truncation=True, max_length=500).to(device)
          with torch.no_grad():
              translated_batch = model.generate(**model_inputs)
          translated += translated_batch
      translated = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
      translated_paragraphs += [" ".join(translated)]
      files.add_paragraph(translated)
  # translated_text = "\n".join(translated_paragraphs)
  
  files.save('New_File.docx')
  return translated_paragraphs


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
btTranslator("/content/drive/MyDrive/13 Gu Family's Wife.docx")
# a1=a.split('\n')
# # print(a1)
# bigtext='''    '''
# for a in a1:
#   # print(a)
#   bigtext=bigtext +' \n '+a

# print(bigtext)



/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1232: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,
